<a href="https://colab.research.google.com/github/Pembo2008/engagement-recognition/blob/main/course_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
from PIL import Image
import cv2
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn import svm,metrics,preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from collections import defaultdict
import os
import random
import numpy as np
from tqdm import tqdm
import time
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
import tensorflow as tf
import torch
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.layers import TimeDistributed, GRU, Dense, Dropout, Flatten, LSTM, Activation, MaxPooling2D
from tensorflow.keras.regularizers import l2 as L2_reg
from tensorflow.keras.layers import Conv2D, BatchNormalization,MaxPool2D, GlobalMaxPool2D, Input, Masking, Conv3D, MaxPooling3D, GlobalMaxPool3D
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.compat.v1.keras.backend import set_session
import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm 
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess=tf.compat.v1.Session(config=config)
set_session(sess)

In [2]:
device = torch.device('cuda')
device

device(type='cuda')

In [3]:
DATA_DIR = '/content/drive/MyDrive/project/Датасэт'

In [ ]:
# def process_dir(dirname):
#     print(dirname)
#     d=os.path.join(DATA_DIR,dirname)
#     for filename in tqdm(os.listdir(d)):
#         if filename.lower().endswith('ini'):
#             continue
#         if os.path.isdir(os.path.join(d,filename)):
#             videofile=None
#             for fn in os.listdir(os.path.join(d,filename)):
#                 if fn.lower().endswith('ini'):
#                     continue
#                 videofile=fn
#             if videofile is None:
#                 print(filename)
#                 continue
#             filename=os.path.join(filename,videofile)
#         fn, ext = os.path.splitext(os.path.basename(filename))
#         outdir=os.path.join(d,fn)
#         if not os.path.exists(outdir):
#             os.makedirs(outdir)
#         command = "ffmpeg -i "+os.path.join(d,filename) + " -r 0.5 '"+outdir+"/%05d.png'"
#         print(command)
#         os.system(command=command)

# process_dir('Test')

Test


  0%|          | 0/18 [00:00<?, ?it/s]

ffmpeg -i /content/drive/MyDrive/project/Датасэт/Test/3_4_2.mp4 -r 0.5 '/content/drive/MyDrive/project/Датасэт/Test/3_4_2/%05d.png'


  6%|▌         | 1/18 [00:15<04:21, 15.39s/it]

ffmpeg -i /content/drive/MyDrive/project/Датасэт/Test/1_4.mp4 -r 0.5 '/content/drive/MyDrive/project/Датасэт/Test/1_4/%05d.png'


...

 89%|████████▉ | 16/18 [08:56<01:07, 33.97s/it]

ffmpeg -i /content/drive/MyDrive/project/Датасэт/Test/2_7_3.mp4 -r 0.5 '/content/drive/MyDrive/project/Датасэт/Test/2_7_3/%05d.png'


 94%|█████████▍| 17/18 [09:52<00:40, 40.67s/it]

ffmpeg -i /content/drive/MyDrive/project/Датасэт/Test/2_7_4.mp4 -r 0.5 '/content/drive/MyDrive/project/Датасэт/Test/2_7_4/%05d.png'


100%|██████████| 18/18 [10:45<00:00, 35.86s/it]


In [ ]:
# process_dir('Train')

Train


  0%|          | 0/20 [00:00<?, ?it/s]

ffmpeg -i /content/drive/MyDrive/project/Датасэт/Train/1_3.mp4 -r 0.5 '/content/drive/MyDrive/project/Датасэт/Train/1_3/%05d.png'


  5%|▌         | 1/20 [00:11<03:46, 11.95s/it]

ffmpeg -i /content/drive/MyDrive/project/Датасэт/Train/0_5_1.mp4 -r 0.5 '/content/drive/MyDrive/project/Датасэт/Train/0_5_1/%05d.png'


...

 90%|█████████ | 18/20 [08:28<01:03, 31.52s/it]

ffmpeg -i /content/drive/MyDrive/project/Датасэт/Train/3_4_7.mp4 -r 0.5 '/content/drive/MyDrive/project/Датасэт/Train/3_4_7/%05d.png'


 95%|█████████▌| 19/20 [09:25<00:39, 39.09s/it]

ffmpeg -i /content/drive/MyDrive/project/Датасэт/Train/0_1.mp4 -r 0.5 '/content/drive/MyDrive/project/Датасэт/Train/0_1/%05d.png'


100%|██████████| 20/20 [09:47<00:00, 29.39s/it]


In [ ]:
# process_dir('Validation')

Validation


  0%|          | 0/20 [00:00<?, ?it/s]

ffmpeg -i /content/drive/MyDrive/project/Датасэт/Validation/3_4.mp4 -r 0.5 '/content/drive/MyDrive/project/Датасэт/Validation/3_4/%05d.png'


  5%|▌         | 1/20 [00:22<07:13, 22.79s/it]

ffmpeg -i /content/drive/MyDrive/project/Датасэт/Validation/2_4.mp4 -r 0.5 '/content/drive/MyDrive/project/Датасэт/Validation/2_4/%05d.png'


...

 90%|█████████ | 18/20 [10:38<01:32, 46.42s/it]

ffmpeg -i /content/drive/MyDrive/project/Датасэт/Validation/1_8.mp4 -r 0.5 '/content/drive/MyDrive/project/Датасэт/Validation/1_8/%05d.png'


 95%|█████████▌| 19/20 [11:26<00:46, 46.82s/it]

ffmpeg -i /content/drive/MyDrive/project/Датасэт/Validation/2_8_1.mp4 -r 0.5 '/content/drive/MyDrive/project/Датасэт/Validation/2_8_1/%05d.png'


100%|██████████| 20/20 [11:54<00:00, 35.74s/it]


In [ ]:
# def extract_openface_features(dirname,outdir):
#     print(dirname)
#     if not os.path.exists(outdir):
#         os.makedirs(outdir)
#     os.chdir(dirname)
#     command='/home/avsavchenko/src/distr/OpenFace/build/bin/FeatureExtraction -pose -aus -gaze -out_dir '+outdir
#     for filename in os.listdir('.'):
#         if filename.lower().endswith('ini'):
#             continue
        
#         if os.path.isdir(filename):
#             command+=' -fdir '+filename
        
#     print(command)
#     os.system(command=command)

# cur_dir=os.getcwd()
# extract_openface_features(os.path.join(DATA_DIR,'frames/validation/'),os.path.join(DATA_DIR,'frames/faces/openface/validation/'))
# extract_openface_features(os.path.join(DATA_DIR,'frames/Train/'),os.path.join(DATA_DIR,'frames/faces/openface/Train/'))
# os.chdir(cur_dir)
# print(os.getcwd())

In [4]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import sys
import os
#os.environ['CUDA_VISIBLE_DEVICES'] = ''
import argparse
import tensorflow as tf
import numpy as np
import cv2
import time

import subprocess, re 


def is_specialfile(path,exts):
    _, file_extension = os.path.splitext(path)
    return file_extension.lower() in exts

img_extensions=['.jpg','.jpeg','.png']
def is_image(path):
    return is_specialfile(path,img_extensions)

video_extensions=['.mov','.avi']
def is_video(path):
    return is_specialfile(path,video_extensions)
    
class FacialImageProcessing:
    # minsize: minimum of faces' size
    def __init__(self, print_stat=False, minsize = 32):
        self.print_stat=print_stat
        self.minsize=minsize
        
        models_path = '/content/drive/MyDrive/project/Датасэт/Models'
        # models_path=os.path.join(models_path,'..','models','pretrained_faces')
        model_files={os.path.join(models_path,'mtcnn.pb'):''}

        with tf.Graph().as_default() as full_graph:
            for model_file in model_files:
                tf.import_graph_def(FacialImageProcessing.load_graph_def(model_file), name=model_files[model_file])
        self.sess=tf.compat.v1.Session(graph=full_graph)#,config=tf.ConfigProto(device_count={'CPU':1,'GPU':0}))
        self.pnet, self.rnet, self.onet = FacialImageProcessing.load_mtcnn(self.sess,full_graph)     

    def close(self):
        self.sess.close()
    
    @staticmethod
    def load_graph_def(frozen_graph_filename):
        graph_def=None
        with tf.io.gfile.GFile(frozen_graph_filename, 'rb') as f:
            graph_def = tf.compat.v1.GraphDef()
            graph_def.ParseFromString(f.read())
        return graph_def
    
    @staticmethod
    def load_graph(frozen_graph_filename, prefix=''):
        graph_def = FacialImageProcessing.load_graph_def(frozen_graph_filename)
        with tf.Graph().as_default() as graph:
            tf.import_graph_def(graph_def, name=prefix)
        return graph

    @staticmethod
    def load_mtcnn(sess,graph):
        pnet_out_1=graph.get_tensor_by_name('pnet/conv4-2/BiasAdd:0')
        pnet_out_2=graph.get_tensor_by_name('pnet/prob1:0')
        pnet_in=graph.get_tensor_by_name('pnet/input:0')
        
        rnet_out_1=graph.get_tensor_by_name('rnet/conv5-2/conv5-2:0')
        rnet_out_2=graph.get_tensor_by_name('rnet/prob1:0')
        rnet_in=graph.get_tensor_by_name('rnet/input:0')
        
        onet_out_1=graph.get_tensor_by_name('onet/conv6-2/conv6-2:0')
        onet_out_2=graph.get_tensor_by_name('onet/conv6-3/conv6-3:0')
        onet_out_3=graph.get_tensor_by_name('onet/prob1:0')
        onet_in=graph.get_tensor_by_name('onet/input:0')
        
        pnet_fun = lambda img : sess.run((pnet_out_1, pnet_out_2), feed_dict={pnet_in:img})
        rnet_fun = lambda img : sess.run((rnet_out_1, rnet_out_2), feed_dict={rnet_in:img})
        onet_fun = lambda img : sess.run((onet_out_1, onet_out_2, onet_out_3), feed_dict={onet_in:img})
        return pnet_fun, rnet_fun, onet_fun
        
    @staticmethod
    def bbreg(boundingbox,reg):
        # calibrate bounding boxes
        if reg.shape[1]==1:
            reg = np.reshape(reg, (reg.shape[2], reg.shape[3]))

        w = boundingbox[:,2]-boundingbox[:,0]+1
        h = boundingbox[:,3]-boundingbox[:,1]+1
        b1 = boundingbox[:,0]+reg[:,0]*w
        b2 = boundingbox[:,1]+reg[:,1]*h
        b3 = boundingbox[:,2]+reg[:,2]*w
        b4 = boundingbox[:,3]+reg[:,3]*h
        boundingbox[:,0:4] = np.transpose(np.vstack([b1, b2, b3, b4 ]))
        return boundingbox
     
    @staticmethod
    def generateBoundingBox(imap, reg, scale, t):
        # use heatmap to generate bounding boxes
        stride=2
        cellsize=12

        imap = np.transpose(imap)
        dx1 = np.transpose(reg[:,:,0])
        dy1 = np.transpose(reg[:,:,1])
        dx2 = np.transpose(reg[:,:,2])
        dy2 = np.transpose(reg[:,:,3])
        y, x = np.where(imap >= t)
        if y.shape[0]==1:
            dx1 = np.flipud(dx1)
            dy1 = np.flipud(dy1)
            dx2 = np.flipud(dx2)
            dy2 = np.flipud(dy2)
        score = imap[(y,x)]
        reg = np.transpose(np.vstack([ dx1[(y,x)], dy1[(y,x)], dx2[(y,x)], dy2[(y,x)] ]))
        if reg.size==0:
            reg = np.empty((0,3))
        bb = np.transpose(np.vstack([y,x]))
        q1 = np.fix((stride*bb+1)/scale)
        q2 = np.fix((stride*bb+cellsize-1+1)/scale)
        boundingbox = np.hstack([q1, q2, np.expand_dims(score,1), reg])
        return boundingbox, reg
     
    # function pick = nms(boxes,threshold,type)
    @staticmethod
    def nms(boxes, threshold, method):
        if boxes.size==0:
            return np.empty((0,3))
        x1 = boxes[:,0]
        y1 = boxes[:,1]
        x2 = boxes[:,2]
        y2 = boxes[:,3]
        s = boxes[:,4]
        area = (x2-x1+1) * (y2-y1+1)
        I = np.argsort(s)
        pick = np.zeros_like(s, dtype=np.int16)
        counter = 0
        while I.size>0:
            i = I[-1]
            pick[counter] = i
            counter += 1
            idx = I[0:-1]
            xx1 = np.maximum(x1[i], x1[idx])
            yy1 = np.maximum(y1[i], y1[idx])
            xx2 = np.minimum(x2[i], x2[idx])
            yy2 = np.minimum(y2[i], y2[idx])
            w = np.maximum(0.0, xx2-xx1+1)
            h = np.maximum(0.0, yy2-yy1+1)
            inter = w * h
            if method == 'Min':
                o = inter / np.minimum(area[i], area[idx])
            else:
                o = inter / (area[i] + area[idx] - inter)
            I = I[np.where(o<=threshold)]
        pick = pick[0:counter]
        return pick

    # function [dy edy dx edx y ey x ex tmpw tmph] = pad(total_boxes,w,h)
    @staticmethod
    def pad(total_boxes, w, h):
        # compute the padding coordinates (pad the bounding boxes to square)
        tmpw = (total_boxes[:,2]-total_boxes[:,0]+1).astype(np.int32)
        tmph = (total_boxes[:,3]-total_boxes[:,1]+1).astype(np.int32)
        numbox = total_boxes.shape[0]

        dx = np.ones((numbox), dtype=np.int32)
        dy = np.ones((numbox), dtype=np.int32)
        edx = tmpw.copy().astype(np.int32)
        edy = tmph.copy().astype(np.int32)

        x = total_boxes[:,0].copy().astype(np.int32)
        y = total_boxes[:,1].copy().astype(np.int32)
        ex = total_boxes[:,2].copy().astype(np.int32)
        ey = total_boxes[:,3].copy().astype(np.int32)

        tmp = np.where(ex>w)
        edx.flat[tmp] = np.expand_dims(-ex[tmp]+w+tmpw[tmp],1)
        ex[tmp] = w
        
        tmp = np.where(ey>h)
        edy.flat[tmp] = np.expand_dims(-ey[tmp]+h+tmph[tmp],1)
        ey[tmp] = h

        tmp = np.where(x<1)
        dx.flat[tmp] = np.expand_dims(2-x[tmp],1)
        x[tmp] = 1

        tmp = np.where(y<1)
        dy.flat[tmp] = np.expand_dims(2-y[tmp],1)
        y[tmp] = 1
        
        return dy, edy, dx, edx, y, ey, x, ex, tmpw, tmph

    # function [bboxA] = rerec(bboxA)
    @staticmethod
    def rerec(bboxA):
        # convert bboxA to square
        h = bboxA[:,3]-bboxA[:,1]
        w = bboxA[:,2]-bboxA[:,0]
        l = np.maximum(w, h)
        bboxA[:,0] = bboxA[:,0]+w*0.5-l*0.5
        bboxA[:,1] = bboxA[:,1]+h*0.5-l*0.5
        bboxA[:,2:4] = bboxA[:,0:2] + np.transpose(np.tile(l,(2,1)))
        return bboxA

    def detect_faces(self,img):
        # im: input image
        # threshold: threshold=[th1 th2 th3], th1-3 are three steps's threshold
        threshold = [ 0.6, 0.7, 0.9 ]  # three steps's threshold
        # fastresize: resize img from last scale (using in high-resolution images) if fastresize==true
        factor = 0.709 # scale factor
        factor_count=0
        total_boxes=np.empty((0,9))
        points=np.array([])
        h=img.shape[0]
        w=img.shape[1]
        minl=np.amin([h, w])
        m=12.0/self.minsize
        minl=minl*m
        # creat scale pyramid
        scales=[]
        while minl>=12:
            scales += [m*np.power(factor, factor_count)]
            minl = minl*factor
            factor_count += 1

        # first stage
        #t=time.time()
        for j in range(len(scales)):
            scale=scales[j]
            hs=int(np.ceil(h*scale))
            ws=int(np.ceil(w*scale))
            im_data = cv2.resize(img, (ws,hs), interpolation=cv2.INTER_AREA)
            im_data = (im_data-127.5)*0.0078125
            img_x = np.expand_dims(im_data, 0)
            img_y = np.transpose(img_x, (0,2,1,3))
            out = self.pnet(img_y)
            out0 = np.transpose(out[0], (0,2,1,3))
            out1 = np.transpose(out[1], (0,2,1,3))
            
            boxes, _ = FacialImageProcessing.generateBoundingBox(out1[0,:,:,1].copy(), out0[0,:,:,:].copy(), scale, threshold[0])
            
            # inter-scale nms
            pick = FacialImageProcessing.nms(boxes.copy(), 0.5, 'Union')
            if boxes.size>0 and pick.size>0:
                boxes = boxes[pick,:]
                total_boxes = np.append(total_boxes, boxes, axis=0)
        numbox = total_boxes.shape[0]
        #elapsed = time.time() - t
        #print('1 phase nb=%d elapsed=%f'%(numbox,elapsed))
        if numbox>0:
            pick = FacialImageProcessing.nms(total_boxes.copy(), 0.7, 'Union')
            total_boxes = total_boxes[pick,:]
            regw = total_boxes[:,2]-total_boxes[:,0]
            regh = total_boxes[:,3]-total_boxes[:,1]
            qq1 = total_boxes[:,0]+total_boxes[:,5]*regw
            qq2 = total_boxes[:,1]+total_boxes[:,6]*regh
            qq3 = total_boxes[:,2]+total_boxes[:,7]*regw
            qq4 = total_boxes[:,3]+total_boxes[:,8]*regh
            total_boxes = np.transpose(np.vstack([qq1, qq2, qq3, qq4, total_boxes[:,4]]))
            total_boxes = FacialImageProcessing.rerec(total_boxes.copy())
            total_boxes[:,0:4] = np.fix(total_boxes[:,0:4]).astype(np.int32)
            dy, edy, dx, edx, y, ey, x, ex, tmpw, tmph = FacialImageProcessing.pad(total_boxes.copy(), w, h)

        numbox = total_boxes.shape[0]
        #elapsed = time.time() - t
        #print('2 phase nb=%d elapsed=%f'%(numbox,elapsed))
        if numbox>0:
            # second stage
            tempimg = np.zeros((24,24,3,numbox))
            for k in range(0,numbox):
                tmp = np.zeros((int(tmph[k]),int(tmpw[k]),3))
                tmp[dy[k]-1:edy[k],dx[k]-1:edx[k],:] = img[y[k]-1:ey[k],x[k]-1:ex[k],:]
                if tmp.shape[0]>0 and tmp.shape[1]>0 or tmp.shape[0]==0 and tmp.shape[1]==0:
                    tempimg[:,:,:,k] = cv2.resize(tmp, (24,24), interpolation=cv2.INTER_AREA)
                else:
                    return np.empty()
            tempimg = (tempimg-127.5)*0.0078125
            tempimg1 = np.transpose(tempimg, (3,1,0,2))
            out = self.rnet(tempimg1)
            out0 = np.transpose(out[0])
            out1 = np.transpose(out[1])
            score = out1[1,:]
            ipass = np.where(score>threshold[1])
            total_boxes = np.hstack([total_boxes[ipass[0],0:4].copy(), np.expand_dims(score[ipass].copy(),1)])
            mv = out0[:,ipass[0]]
            if total_boxes.shape[0]>0:
                pick = FacialImageProcessing.nms(total_boxes, 0.7, 'Union')
                total_boxes = total_boxes[pick,:]
                total_boxes = FacialImageProcessing.bbreg(total_boxes.copy(), np.transpose(mv[:,pick]))
                total_boxes = FacialImageProcessing.rerec(total_boxes.copy())

        numbox = total_boxes.shape[0]
        #elapsed = time.time() - t
        #print('3 phase nb=%d elapsed=%f'%(numbox,elapsed))
        if numbox>0:
            # third stage
            total_boxes = np.fix(total_boxes).astype(np.int32)
            dy, edy, dx, edx, y, ey, x, ex, tmpw, tmph = FacialImageProcessing.pad(total_boxes.copy(), w, h)
            tempimg = np.zeros((48,48,3,numbox))
            for k in range(0,numbox):
                tmp = np.zeros((int(tmph[k]),int(tmpw[k]),3))
                tmp[dy[k]-1:edy[k],dx[k]-1:edx[k],:] = img[y[k]-1:ey[k],x[k]-1:ex[k],:]
                if tmp.shape[0]>0 and tmp.shape[1]>0 or tmp.shape[0]==0 and tmp.shape[1]==0:
                    tempimg[:,:,:,k] = cv2.resize(tmp, (48,48), interpolation=cv2.INTER_AREA)
                else:
                    return np.empty()
            tempimg = (tempimg-127.5)*0.0078125
            tempimg1 = np.transpose(tempimg, (3,1,0,2))
            out = self.onet(tempimg1)
            out0 = np.transpose(out[0])
            out1 = np.transpose(out[1])
            out2 = np.transpose(out[2])
            score = out2[1,:]
            points = out1
            ipass = np.where(score>threshold[2])
            points = points[:,ipass[0]]
            total_boxes = np.hstack([total_boxes[ipass[0],0:4].copy(), np.expand_dims(score[ipass].copy(),1)])
            mv = out0[:,ipass[0]]

            w = total_boxes[:,2]-total_boxes[:,0]+1
            h = total_boxes[:,3]-total_boxes[:,1]+1
            points[0:5,:] = np.tile(w,(5, 1))*points[0:5,:] + np.tile(total_boxes[:,0],(5, 1))-1
            points[5:10,:] = np.tile(h,(5, 1))*points[5:10,:] + np.tile(total_boxes[:,1],(5, 1))-1
            if total_boxes.shape[0]>0:
                total_boxes = FacialImageProcessing.bbreg(total_boxes.copy(), np.transpose(mv))
                pick = FacialImageProcessing.nms(total_boxes.copy(), 0.7, 'Min')
                total_boxes = total_boxes[pick,:]
                points = points[:,pick]
        #elapsed = time.time() - t
        #print('4 phase elapsed=%f'%(elapsed))            
        return total_boxes, points

In [5]:
imgProcessing=FacialImageProcessing(False)

In [ ]:
INPUT_SIZE = (224,224)
def save_faces(source_path,save_path):
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    for folder in tqdm(os.listdir(source_path)):
        if not os.path.exists(os.path.join(save_path, folder)):
            os.mkdir(os.path.join(save_path, folder))

            for image in os.listdir(os.path.join(source_path, folder)):
                filename = os.path.join(source_path, folder, image)
                frame_bgr = cv2.imread(filename)
                frame = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2RGB)
                bounding_boxes, _ = imgProcessing.detect_faces(frame)

                if len(bounding_boxes)==0:
                    print('No faces found for ',filename)
                    face_img = frame_bgr
                    faceFound='noface'
                else:
                    if len(bounding_boxes)>1:
                        print('Too many faces (',len(bounding_boxes),') found for ',filename)
                        bounding_boxes=bounding_boxes[:1]

                    b=[int(bi) for bi in bounding_boxes[0]]
                    x1,y1,x2,y2=b[0:4]
                    face_img=frame_bgr[y1:y2,x1:x2,:]

                    if np.prod(face_img.shape)==0:
                        print('Empty face ',b,' found for ',filename)
                        continue
                    
                    faceFound=''

                #face_img=cv2.resize(face_img,INPUT_SIZE)
                root,ext=os.path.splitext(image)
                cv2.imwrite(os.path.join(save_path, folder, root+faceFound+ext), face_img) 
        else:
            print(folder)
        
save_faces(os.path.join(DATA_DIR,'Validation/frames'),os.path.join(DATA_DIR,'cropped/Validation'))

  0%|          | 0/20 [00:00<?, ?it/s]

3_4


 15%|█▌        | 3/20 [02:03<11:12, 39.59s/it]

Too many faces ( 2 ) found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/3_1_1/00024.png
Too many faces ( 2 ) found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/3_1_1/00025.png


 20%|██        | 4/20 [02:42<10:30, 39.42s/it]

...

 75%|███████▌  | 15/20 [12:16<04:57, 59.41s/it]

No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00029.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00030.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00031.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00032.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00033.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00034.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00035.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00036.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00037.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00038.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/

 80%|████████  | 16/20 [12:42<03:17, 49.44s/it]

No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00016.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00017.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00018.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00019.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00020.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00021.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00022.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00023.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00046.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00047.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/

100%|██████████| 20/20 [18:13<00:00, 54.69s/it]


In [ ]:
save_faces(os.path.join(DATA_DIR,'Train/frames'),os.path.join(DATA_DIR,'cropped/Train'))

  0%|          | 0/20 [00:00<?, ?it/s]

No faces found for  /content/drive/MyDrive/project/Датасэт/Train/frames/1_3/00009.png


  5%|▌         | 1/20 [00:35<11:08, 35.19s/it]

No faces found for  /content/drive/MyDrive/project/Датасэт/Train/frames/0_5_1/00040.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Train/frames/0_5_1/00041.png


 20%|██        | 4/20 [02:35<09:56, 37.27s/it]

...

 80%|████████  | 16/20 [15:52<05:02, 75.69s/it]

Too many faces ( 2 ) found for  /content/drive/MyDrive/project/Датасэт/Train/frames/2_4_2/00079.png
Too many faces ( 2 ) found for  /content/drive/MyDrive/project/Датасэт/Train/frames/2_4_2/00082.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Train/frames/2_4_2/00120.png
Too many faces ( 2 ) found for  /content/drive/MyDrive/project/Датасэт/Train/frames/2_4_2/00141.png


 95%|█████████▌| 19/20 [18:23<00:59, 59.54s/it]

No faces found for  /content/drive/MyDrive/project/Датасэт/Train/frames/0_1/00014.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Train/frames/0_1/00015.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Train/frames/0_1/00016.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Train/frames/0_1/00047.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Train/frames/0_1/00048.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Train/frames/0_1/00049.png


100%|██████████| 20/20 [19:28<00:00, 58.42s/it]


In [ ]:
save_faces(os.path.join(DATA_DIR,'Test/frames'),os.path.join(DATA_DIR,'cropped/Test'))

 56%|█████▌    | 10/18 [07:39<04:21, 32.65s/it]

No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/1_6_1/00008.png


 67%|██████▋   | 12/18 [09:53<05:06, 51.02s/it]

No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/1_7/00046.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/1_7/00047.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/1_7/00048.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/1_7/00049.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/1_7/00050.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/1_7/00051.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/1_7/00052.png


 72%|███████▏  | 13/18 [10:48<04:21, 52.37s/it]

...

 83%|████████▎ | 15/18 [11:44<02:00, 40.07s/it]

No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/3_7/00005.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/3_7/00046.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/3_7/00047.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/3_7/00048.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/3_7/00049.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/3_7/00060.png


 89%|████████▉ | 16/18 [12:46<01:33, 46.75s/it]

No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/2_7_3/00059.png


100%|██████████| 18/18 [13:43<00:00, 45.78s/it]


In [6]:
def get_iou(bb1, bb2):
    """
    Calculate the Intersection over Union (IoU) of two bounding boxes.

    Parameters
    ----------
    bb1 : array
        order: {'x1', 'y1', 'x2', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner
    bb2 : array
        order: {'x1', 'y1', 'x2', 'y2'}
        The (x1, y1) position is at the top left corner,
        the (x2, y2) position is at the bottom right corner

    Returns
    -------
    float
        in [0, 1]
    """

    # determine the coordinates of the intersection rectangle
    x_left = max(bb1[0], bb2[0])
    y_top = max(bb1[1], bb2[1])
    x_right = min(bb1[2], bb2[2])
    y_bottom = min(bb1[3], bb2[3])

    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # The intersection of two axis-aligned bounding boxes is always an
    # axis-aligned bounding box
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # compute the area of both AABBs
    bb1_area = (bb1[2] - bb1[0]) * (bb1[3] - bb1[1])
    bb2_area = (bb2[2] - bb2[0]) * (bb2[3] - bb2[1])

    # compute the intersection over union by taking the intersection
    # area and dividing it by the sum of prediction + ground-truth
    # areas - the interesection area
    iou = intersection_area / float(bb1_area + bb2_area - intersection_area)
    return iou
def preprocess(img, bbox=None, landmark=None, **kwargs):
    M = None
    image_size = [224,224]
    src = np.array([
      [30.2946, 51.6963],
      [65.5318, 51.5014],
      [48.0252, 71.7366],
      [33.5493, 92.3655],
      [62.7299, 92.2041] ], dtype=np.float32 )
    if image_size[1]==224:
        src[:,0] += 8.0
    src*=2
    if landmark is not None:
        dst = landmark.astype(np.float32)

        tform = trans.SimilarityTransform()
        #dst=dst[:3]
        #src=src[:3]
        #print(dst.shape,src.shape,dst,src)
        tform.estimate(dst, src)
        M = tform.params[0:2,:]
        #M = cv2.estimateRigidTransform( dst.reshape(1,5,2), src.reshape(1,5,2), False)
        #print(M)

    if M is None:
        if bbox is None: #use center crop
            det = np.zeros(4, dtype=np.int32)
            det[0] = int(img.shape[1]*0.0625)
            det[1] = int(img.shape[0]*0.0625)
            det[2] = img.shape[1] - det[0]
            det[3] = img.shape[0] - det[1]
        else:
              det = bbox
        margin = kwargs.get('margin', 44)
        bb = np.zeros(4, dtype=np.int32)
        bb[0] = np.maximum(det[0]-margin//2, 0)
        bb[1] = np.maximum(det[1]-margin//2, 0)
        bb[2] = np.minimum(det[2]+margin//2, img.shape[1])
        bb[3] = np.minimum(det[3]+margin//2, img.shape[0])
        ret = img[bb[1]:bb[3],bb[0]:bb[2],:]
        if len(image_size)>0:
              ret = cv2.resize(ret, (image_size[1], image_size[0]))
        return ret 
    else: #do align using landmark
        assert len(image_size)==2
        warped = cv2.warpAffine(img,M,(image_size[1],image_size[0]), borderValue = 0.0)
        return warped

In [ ]:
def save_aligned_faces(source_path,save_path):
    if not os.path.exists(save_path):
        os.makedirs(save_path)

    for folder in tqdm(os.listdir(source_path)):
        if not os.path.exists(os.path.join(save_path, folder)):
            os.mkdir(os.path.join(save_path, folder))

            prev_b = None
            counter = 0

            for image in sorted(os.listdir(os.path.join(source_path, folder))):
                filename = os.path.join(source_path, folder, image)
                frame = cv2.imread(filename)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                bounding_boxes, points = imgProcessing.detect_faces(frame)
                points = points.T

                best_ind=None
                if len(bounding_boxes)==0:
                    print('No faces found for ',filename)
                    counter+=1
                    if prev_b is None or counter>3:
                        continue
                    else:
                        b=prev_b
                elif len(bounding_boxes)>1:
                    print('Too many faces (',len(bounding_boxes),') found for ',filename)
                    if prev_b is None:
                        #continue
                        best_ind=0
                        b=[int(bi) for bi in bounding_boxes[best_ind]]
                        counter=0
                    else:
                        best_iou=0
                        for i in range(len(bounding_boxes)):
                            iou=get_iou(bounding_boxes[i],prev_b)
                            if iou>best_iou:
                                best_iou=iou
                                best_ind=i
                        if best_iou>0:
                            b=[int(bi) for bi in bounding_boxes[best_ind]]
                            print('best_iou (',best_iou,') best_bb ',bounding_boxes[best_ind])
                        else:
                            #continue
                            best_ind=0
                            b=[int(bi) for bi in bounding_boxes[best_ind]]
                            counter=0
                else:
                    best_ind=0
                    b=[int(bi) for bi in bounding_boxes[best_ind]]
                    counter=0
                prev_b=b

                if True:
                    p=None
                    if best_ind is not None:
                        p=points[best_ind]
                        if True: #not USE_RETINA_FACE:
                            p = p.reshape((2,5)).T
                    face_img=preprocess(frame,b,None) #p)
                else:
                    x1,y1,x2,y2=b[0:4]
                    face_img=frame[y1:y2,x1:x2,:]
                if np.prod(face_img.shape)==0:
                    print('Empty face ',b,' found for ',filename)
                    continue

                cv2.imwrite(os.path.join(save_path, folder, image), cv2.cvtColor(face_img, cv2.COLOR_RGB2BGR)) 


In [ ]:
save_aligned_faces(os.path.join(DATA_DIR,'Test/frames'),os.path.join(DATA_DIR,'cropped/aligned/Test'))
save_aligned_faces(os.path.join(DATA_DIR,'Train/frames'),os.path.join(DATA_DIR,'cropped/aligned/Train'))
save_aligned_faces(os.path.join(DATA_DIR,'Validation/frames'),os.path.join(DATA_DIR,'cropped/aligned/Validation'))

 56%|█████▌    | 10/18 [03:21<02:38, 19.79s/it]

No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/1_6_1/00008.png


 67%|██████▋   | 12/18 [03:39<01:25, 14.27s/it]

No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/1_7/00046.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/1_7/00047.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/1_7/00048.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/1_7/00049.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/1_7/00050.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Test/frames/1_7/00051.png


 72%|███████▏  | 13/18 [03:46<01:01, 12.24s/it]

...

 75%|███████▌  | 15/20 [03:32<00:55, 11.17s/it]

No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00029.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00030.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00031.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00032.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00033.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00034.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00035.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00036.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00037.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_5/00038.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/

 80%|████████  | 16/20 [03:54<00:57, 14.46s/it]

No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00016.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00017.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00018.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00019.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00020.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00021.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00022.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00023.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00046.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/frames/2_7_6/00047.png
No faces found for  /content/drive/MyDrive/project/Датасэт/Validation/

100%|██████████| 20/20 [04:58<00:00, 14.94s/it]


In [7]:
from tensorflow.keras.applications import mobilenet
from tensorflow.keras.models import load_model,Model

In [8]:
def mobilenet_preprocess_input(x,**kwargs):
    x[..., 0] -= 103.939
    x[..., 1] -= 116.779
    x[..., 2] -= 123.68
    return x

preprocessing_function=mobilenet_preprocess_input

In [ ]:
model_name='mobilenet_7'
base_model=load_model('/content/drive/MyDrive/project/Датасэт/Models/'+model_name+'.h5')
feature_extractor_model=Model(base_model.input,[base_model.get_layer('global_pooling').output,base_model.get_layer('feats').output,base_model.output])
feature_extractor_model.summary()
_,w,h,_=feature_extractor_model.input.shape
print(w,h)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 conv1_pad (ZeroPadding2D)   (None, 225, 225, 3)       0         
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288       
                                                             

In [ ]:
feature_extractor_model.output

[<KerasTensor: shape=(None, 1024) dtype=float32 (created by layer 'global_pooling')>,
 <KerasTensor: shape=(None, 256) dtype=float32 (created by layer 'feats')>,
 <KerasTensor: shape=(None, 7) dtype=float32 (created by layer 'emotion_preds')>]

In [ ]:
def get_features_scores(data_dir):
    filename2features={}
    for filename in tqdm(os.listdir(data_dir)):
        frames_dir=os.path.join(data_dir,filename)
        X_global_features,X_feats,X_scores,X_isface=[],[],[],[]
        imgs=[]
        faces_only=[fn for fn in os.listdir(frames_dir) if 'noface' not in fn]
        for img_name in sorted(faces_only):
            img=cv2.imread(os.path.join(frames_dir,img_name))
            X_isface.append('noface' not in img_name)
                
            if img.size:
                img=cv2.resize(img,(w,h))
                img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
                imgs.append(img)
                if len(imgs)>=512:        
                    #inp = preprocessing_function(np.expand_dims(img, axis=0).astype(np.float32))
                    inp = preprocessing_function(np.array(imgs, dtype=np.float32))
                    global_features,feats,scores=feature_extractor_model.predict(inp)
                    #print(global_features.shape,feats.shape,scores.shape)
                    if len(X_feats)==0:
                        X_feats=feats
                        X_global_features=global_features
                        X_scores=scores
                    else:
                        X_feats=np.concatenate((X_feats,feats),axis=0)
                        X_global_features=np.concatenate((X_global_features,global_features),axis=0)
                        X_scores=np.concatenate((X_scores,scores),axis=0)
                    
                    imgs=[]

        if len(imgs)>0:        
            inp = preprocessing_function(np.array(imgs, dtype=np.float32))
            global_features,feats,scores=feature_extractor_model.predict(inp)
            #print(global_features.shape,feats.shape,scores.shape)
            if len(X_feats)==0:
                X_feats=feats
                X_global_features=global_features
                X_scores=scores
            else:
                X_feats=np.concatenate((X_feats,feats),axis=0)
                X_global_features=np.concatenate((X_global_features,global_features),axis=0)
                X_scores=np.concatenate((X_scores,scores),axis=0)

        X_isface=np.array(X_isface)
        #print(X_global_features.shape,X_feats.shape,X_scores.shape)
        filename2features[filename]=(X_global_features,X_feats,X_scores,X_isface)
    return filename2features

filename2features_train=get_features_scores(os.path.join(DATA_DIR,'cropped/aligned/Train')) #mtcnn_aligned
filename2features_val=get_features_scores(os.path.join(DATA_DIR,'cropped/aligned/Validation')) #_cropped
filename2features_test=get_features_scores(os.path.join(DATA_DIR,'cropped/aligned/Test')) #_cropped

100%|██████████| 18/18 [04:01<00:00, 13.41s/it]


In [14]:
pip install timm==0.4.5

     |████████████████████████████████| 287 kB 5.6 MB/s 


In [15]:
import timm

In [16]:
if False:
    model_name='enet_b2_8'
    IMG_SIZE=260 #224 #
else:
    model_name='enet_b0_8_best_afew'
    IMG_SIZE=224
PATH=DATA_DIR+'/Models/'+model_name+'.pt'
test_transforms = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)
np_transforms = transforms.Compose(
    [
        transforms.ToPILImage(None),
        transforms.Resize((IMG_SIZE,IMG_SIZE)),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
    ]
)

feature_extractor_model = torch.load(PATH)

In [17]:
feature_extractor_model.classifier=torch.nn.Identity()
feature_extractor_model.eval()

EfficientNet(
  (conv_stem): Conv2dSame(3, 32, kernel_size=(3, 3), stride=(2, 2), bias=False)
  (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  (act1): SiLU(inplace=True)
  (blocks): Sequential(
    (0): Sequential(
      (0): DepthwiseSeparableConv(
        (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
        (bn1): BatchNorm2d(32, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act1): SiLU(inplace=True)
        (se): SqueezeExcite(
          (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
          (act1): SiLU(inplace=True)
          (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
        )
        (conv_pw): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
        (act2): Identity()
      )
    )
    (1): Sequential(
      (0

In [18]:
# emotion_ind2label={0: 'Not-Engaget', 1: 'Barely-Engaged', 2: 'Engaged', 3: 'Highly-Engaged'}

In [19]:
def get_features(data_dir):
    filename2features={}
    for filename in tqdm(os.listdir(data_dir)):
        frames_dir=os.path.join(data_dir,filename)
        X_global_features=[]
        imgs=[]
        faces_only=[fn for fn in os.listdir(frames_dir) if 'noface' not in fn]
        for img_name in sorted(faces_only):
            img = Image.open(os.path.join(frames_dir,img_name))
            img_tensor = test_transforms(img)
            if img.size:
                imgs.append(img_tensor)
                if len(imgs)>=64:        
                    features = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
                    features=features.data.cpu().numpy()
                    #print(features.shape)

                    if len(X_global_features)==0:
                        X_global_features=features
                    else:
                        X_global_features=np.concatenate((X_global_features,features),axis=0)

                    imgs=[]
            

        if len(imgs)>0:        
            features = feature_extractor_model(torch.stack(imgs, dim=0).to(device))
            features=features.data.cpu().numpy()
            #print(features.shape)

            if len(X_global_features)==0:
                X_global_features=features
            else:
                X_global_features=np.concatenate((X_global_features,features),axis=0)

            imgs=[]

        #print(X_global_features.shape,X_feats.shape,X_scores.shape)
        filename2features[filename]=X_global_features
    return filename2features

print(test_transforms)
filename2features_val=get_features(os.path.join(DATA_DIR,'cropped/aligned/Validation/'))

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


  0%|          | 0/20 [00:00<?, ?it/s]

In [21]:
filename2features_train=get_features(os.path.join(DATA_DIR,'cropped/aligned/Train/'))
filename2features_test=get_features(os.path.join(DATA_DIR,'cropped/aligned/Test/'))

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
# torch.cuda.empty_cache()

In [38]:
import csv

def create_dataset(filename2features):
    x = []
    y = []
    ind=0
    for fn in filename2features:
        cur_features=filename2features[fn]
        total_features=None
        total_features=np.std(cur_features, axis=0)
        x.append(total_features)
        y.append(video2label[fn])
    x=np.array(x)
    y=np.array(y)
    print(x.shape,y.shape)
    return x,y

video2label={}
with open(os.path.join('/content/drive/MyDrive/project/Датасэт/Labels/','ValidationLabels.csv'), mode='r') as csvfile:
    labels_reader = csv.reader(csvfile, delimiter=';')
    for i,row in enumerate(labels_reader):
        if i==0:
            #print('first:',row)
            continue
        videoname,label=row[0].split('.')[0],float(row[1])
        video2label[videoname]=label

x_val, y_val = create_dataset(filename2features_val)

(20, 1280) (20,)


In [41]:
video2label={}
with open(os.path.join('/content/drive/MyDrive/project/Датасэт/Labels/','TrainLabels.csv'), mode='r') as csvfile:
    labels_reader = csv.reader(csvfile, delimiter=';')
    for i,row in enumerate(labels_reader):
        if i==0:
            #print('first:',row)
            continue
        videoname,label=row[0].split('.')[0],float(row[1])
        video2label[videoname]=label

x_train, y_train = create_dataset(filename2features_train)

(20, 1280) (20,)


In [49]:
video2label={}
with open(os.path.join('/content/drive/MyDrive/project/Датасэт/Labels/','TestLabels.csv'), mode='r') as csvfile:
    labels_reader = csv.reader(csvfile, delimiter=';')
    for i,row in enumerate(labels_reader):
        if i==0:
            #print('first:',row)
            continue
        videoname,label=row[0].split('.')[0],float(row[1])
        video2label[videoname]=label

x_test, y_test = create_dataset(filename2features_test)

(18, 1280) (18,)


In [50]:
reg_coef,reg_intercept = np.load('/content/drive/MyDrive/project/Датасэт/Models/mord_reg_enet_b0_8_best_afew.npy',allow_pickle=True)

In [52]:
y_test

array([3., 1., 3., 3., 3., 0., 3., 1., 2., 2., 1., 2., 1., 2., 1., 3., 2.,
       2.])

In [53]:
y_pred=np.dot(x_test,reg_coef)+reg_intercept
y_pred = np.round(y_pred)
y_pred = np.clip(y_pred, 0, 3)
print(y_pred)
int2float={0:0,1:0.33,2:0.66,3:1} #EngageWild: disengaged, barely engaged, engaged, highly engaged.
y_pred_float=np.array([int2float[y] for y in y_pred])
print(y_pred_float)
print("MSE:",((y_test-y_pred_float)**2).mean())

[3. 1. 2. 2. 2. 2. 2. 2. 2. 2. 0. 2. 1. 1. 0. 1. 1. 1.]
[1.   0.33 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.   0.66 0.33 0.33
 0.   0.33 0.33 0.33]
MSE: 2.790766666666666


In [55]:
y_train

array([1., 0., 2., 2., 3., 3., 2., 3., 1., 0., 1., 2., 1., 0., 1., 2., 2.,
       1., 3., 0.])

In [57]:
y_pred=np.dot(x_train,reg_coef)+reg_intercept
y_pred = np.round(y_pred)
y_pred = np.clip(y_pred, 0, 3)
print(y_pred)
int2float={0:0,1:0.33,2:0.66,3:1} #EngageWild: disengaged, barely engaged, engaged, highly engaged.
y_pred_float=np.array([int2float[y] for y in y_pred])
print(y_pred_float)
print("MSE:",((y_train-y_pred_float)**2).mean())

[0. 0. 3. 2. 3. 3. 2. 2. 2. 2. 2. 2. 2. 2. 1. 2. 2. 2. 2. 1.]
[0.   0.   1.   0.66 1.   1.   0.66 0.66 0.66 0.66 0.66 0.66 0.66 0.66
 0.33 0.66 0.66 0.66 0.66 0.33]
MSE: 1.5910299999999997


In [58]:
y_val

array([3., 2., 3., 3., 0., 1., 2., 3., 2., 1., 3., 2., 3., 0., 1., 2., 2.,
       0., 1., 2.])

In [59]:
y_pred=np.dot(x_val,reg_coef)+reg_intercept
y_pred = np.round(y_pred)
y_pred = np.clip(y_pred, 0, 3)
print(y_pred)
int2float={0:0,1:0.33,2:0.66,3:1} #EngageWild: disengaged, barely engaged, engaged, highly engaged.
y_pred_float=np.array([int2float[y] for y in y_pred])
print(y_pred_float)
print("MSE:",((y_val-y_pred_float)**2).mean())

[2. 2. 3. 3. 1. 1. 2. 2. 3. 2. 3. 2. 2. 1. 1. 1. 1. 2. 2. 3.]
[0.66 0.66 1.   1.   0.33 0.33 0.66 0.66 1.   0.66 1.   0.66 0.66 0.33
 0.33 0.33 0.33 0.66 0.66 1.  ]
MSE: 2.1586899999999996


In [61]:
y_test

array([3., 1., 3., 3., 3., 0., 3., 1., 2., 2., 1., 2., 1., 2., 1., 3., 2.,
       2.])

In [65]:
!pip install pickle5
import pickle5 as pickle

     |████████████████████████████████| 256 kB 5.2 MB/s 


In [66]:
with open('/content/drive/MyDrive/project/Датасэт/Models/daisee_svm_norm_enet_b0_8_best_afew.pkl', 'rb') as handle:
    cls=pickle.load(handle)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:338: UserWarning: Trying to unpickle estimator SVC from version 0.24.2 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [67]:
x_test_norm=preprocessing.normalize(x_test,norm='l2')
y_pred = cls.predict(x_test_norm)
print(y_pred)
int2float={0:0,1:0.33,2:0.66,3:1} # very low, low, high, very high
y_pred_float=np.array([int2float[y] for y in y_pred])
print("MSE:",((y_test-y_pred_float)**2).mean())

[3 2 2 2 2 3 2 2 2 2 2 2 2 2 2 2 2 2]
MSE: 2.4294222222222217
